Download GNPS and <i>in silico</i> structural annotation (NAP, Dereplicator, Varquest) data <br>
Replace Job IDs below with your GNPS, Varquest or Dereplicator job IDs:

In [1]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/result.jsp?task=8c6f75c33ce449189236ffdf929e5589&view=cytoscape_preview' -o GNPS_output_graphML.zip
! unzip -d GNPS_output_graphML/GNPS_output_graphML.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: 'https
'view'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
UnZip 6.00 of 20 April 2009, by Info-ZIP.  Maintained by C. Spieler.  Send
bug reports using http://www.info-zip.org/zip-bug.html; see README for details.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T

load libraries

In [7]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/usr/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [9]:
netfile = 'GNPS_output/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
gnpslibfile = 'GNPS_output/result_specnets_DB/'+ str(os.listdir('GNPS_output/result_specnets_DB/')[0])

nap_id = '675a7abda69b498b883ffce2d2b076dc'

nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')
matches = [gnpslib, nap]

In [10]:
out = unique_smiles(matches)

In [11]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [12]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt

In [13]:
ikeys = pd.read_csv("InchiKeys.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys.txt", quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [14]:
get_classifications("InchiKeys.txt")

1000


In [15]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [16]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,1,22,6,Organic compounds,0.833333,Lipids and lipid-like molecules,0.181878,Prenol lipids,0.116402,NaN,0.268849,Hybrid peptides,0.0833333,Aromatic heteropolycyclic compounds,0.548942
1,8,22,6,Organic compounds,0.833333,Lipids and lipid-like molecules,0.181878,Prenol lipids,0.116402,NaN,0.268849,Hybrid peptides,0.0833333,Aromatic heteropolycyclic compounds,0.548942
2,9,22,6,Organic compounds,0.833333,Lipids and lipid-like molecules,0.181878,Prenol lipids,0.116402,NaN,0.268849,Hybrid peptides,0.0833333,Aromatic heteropolycyclic compounds,0.548942
3,179,22,6,Organic compounds,0.833333,Lipids and lipid-like molecules,0.181878,Prenol lipids,0.116402,NaN,0.268849,Hybrid peptides,0.0833333,Aromatic heteropolycyclic compounds,0.548942
4,219,22,6,Organic compounds,0.833333,Lipids and lipid-like molecules,0.181878,Prenol lipids,0.116402,NaN,0.268849,Hybrid peptides,0.0833333,Aromatic heteropolycyclic compounds,0.548942


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [17]:
final.to_csv("ClassyFireResults_Network.txt", sep = '\t', index = False)

create graphML file

In [18]:
graphMLfile = 'GNPS_output/' + [x for x in os.listdir('GNPS_output/') if 'METABOLOMICS' in x][0]

In [19]:
graphML = read_graphml(graphMLfile)

In [20]:
graphML_classy = make_classyfire_graphml(graphML,final)

write graphML file

In [21]:
nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)